# Quantum Simulation Demo - Modular Approach

Notebook này demo cách sử dụng code được tổ chức thành các module riêng biệt.

## Cấu trúc module:
- `config.py`: Chứa các constants và configuration
- `quantum_setup.py`: Setup các operator quantum và hamiltonians
- `ansatz.py`: Chứa các hàm ansatz cho PennyLane
- `time_evolution.py`: Tính toán time evolution
- `optimization.py`: VQE và quantum compilation optimization
- `visualization.py`: Vẽ biểu đồ
- `utils.py`: Utility functions
- `main_workflow.py`: Orchestrate toàn bộ workflow

## 1. Import các module cần thiết

In [2]:
# Import modules
from quantum_setup import setup_molecular_problem, setup_qubit_operators
from time_evolution import compute_target_unitaries
from vqe_optimization import run_vqe, get_vqe_ground_state
from vqc_optimization import run_vqa_time_evolution
from visualization import plot_optimization_convergence
import config
import numpy as np

## 2. Kiểm tra configuration

In [3]:
# Xem các config parameters
print("Configuration Parameters:")
print(f"Molecule: {config.MOLECULE_GEOMETRY}")
print(f"Basis set: {config.BASIS_SET}")
print(f"E0: {config.E0}")
print(f"Gamma: {config.GAMMA}")
print(f"Optimizer max iterations: {config.OPTIMIZER_MAXITER}")
print(f"Time range: {config.TIME_RANGE}")
print(f"Number of time points: {config.NUM_TIME_POINTS}")

Configuration Parameters:
Molecule: H 0 0 0; H 0 0 2.0
Basis set: sto3g
E0: 0.01
Gamma: 0.25
Optimizer max iterations: 300
Time range: [0, 300]
Number of time points: 50


## 3. Setup molecular problem

In [4]:
# Setup molecular problem
print("Setting up molecular problem...")
problem, hamiltonian, dipole = setup_molecular_problem()

print(f"Number of qubits: {problem.num_spin_orbitals}")
print(f"Number of particles: {problem.num_particles}")
print(f"Number of spatial orbitals: {problem.num_spatial_orbitals}")

Setting up molecular problem...
Number of qubits: 4
Number of particles: (1, 1)
Number of spatial orbitals: 2
Number of qubits: 4
Number of particles: (1, 1)
Number of spatial orbitals: 2


## 4. Setup qubit operators

In [5]:
# Setup qubit operators
print("Setting up qubit operators...")
qubit_ops = setup_qubit_operators(hamiltonian, dipole)

H_0 = qubit_ops['H_0']
H_static = qubit_ops['H_static']
dipole_qubit = qubit_ops['dipole_qubit']
dipole_matrix = qubit_ops['dipole_matrix']
mapper = qubit_ops['mapper']

print(f"Hamiltonian shape: {H_static.shape}")
print(f"Number of qubits: {H_0.num_qubits}")
print(f"Number of Pauli terms: {len(H_0.paulis)}")

Setting up qubit operators...
Hamiltonian shape: (16, 16)
Number of qubits: 4
Number of Pauli terms: 15


## 5. Visualize Electric Field

In [ ]:
# Vẽ electric field
times = np.linspace(-50, 50, 200)
plot_electric_field(times, "Electric Field Profile")

## 6. Run VQE để tìm ground state

In [6]:
# Setup ansatz
ansatz = setup_ansatz(problem, mapper)
print(f"Ansatz has {ansatz.num_parameters} parameters")

# Run VQE
print("Running VQE...")
vqe_result = run_vqe(H_0, ansatz)

print(f"VQE ground state energy: {vqe_result.optimal_value:.6f}")
print(f"Number of function evaluations: {vqe_result.optimizer_evals}")

# Get ground state
psi_0_vqe = get_vqe_ground_state(ansatz, vqe_result.optimal_parameters)
print(f"Ground state norm: {np.linalg.norm(psi_0_vqe):.6f}")

NameError: name 'setup_ansatz' is not defined

## 7. Compute target unitaries for time evolution

In [ ]:
# Compute target unitaries
print("Computing target unitaries...")
target_unitaries, times_training = compute_target_unitaries(H_static, dipole_matrix)

print(f"Computed {len(target_unitaries)} target unitaries")
print(f"Time range: [{times_training[0]:.1f}, {times_training[-1]:.1f}]")
print(f"Unitary matrix shape: {target_unitaries[0].shape}")

## 8. Example: Check unitary properties

In [ ]:
# Kiểm tra tính chất unitary của một vài ma trận
def check_unitarity(U, tolerance=1e-10):
    """Check if matrix is unitary"""
    U_dag = np.conj(U.T)
    identity = U @ U_dag
    error = np.max(np.abs(identity - np.eye(U.shape[0])))
    return error < tolerance, error

print("Checking unitarity of target matrices:")
for i, U in enumerate(target_unitaries[:5]):  # Check first 5
    is_unitary, error = check_unitarity(U)
    print(f"Time {times_training[i]:.1f}: Unitary = {is_unitary}, Error = {error:.2e}")

## 9. Save results

In [ ]:
# Prepare results dictionary
results = {
    'vqe_energy': vqe_result.optimal_value,
    'optimal_parameters': vqe_result.optimal_parameters,
    'ground_state': psi_0_vqe,
    'times': times_training,
    'num_qubits': H_0.num_qubits,
    'problem_info': {
        'num_particles': problem.num_particles,
        'num_spatial_orbitals': problem.num_spatial_orbitals
    }
}

# Save results
save_results(results, 'demo_results.json')
print("Results saved successfully!")

## 10. Chạy complete workflow (Alternative approach)

In [ ]:
# Alternatively, bạn có thể chạy toàn bộ workflow một lần
print("Running complete workflow...")
complete_results = run_complete_workflow()

print("\nWorkflow Summary:")
print(f"VQE Energy: {complete_results['vqe_energy']:.6f}")
print(f"Exact Energy: {complete_results['exact_energy']:.6f}")
print(f"Energy Error: {complete_results['energy_error']:.6f}")
print(f"Number of target unitaries: {len(complete_results['target_unitaries'])}")

## 11. System information

In [ ]:
# Print system information
print_system_info()

## Kết luận

Notebook này demo cách:
1. Tổ chức code thành các module riêng biệt
2. Import và sử dụng functions từ các module
3. Chạy từng bước riêng lẻ hoặc toàn bộ workflow
4. Visualize kết quả
5. Save/load data

### Ưu điểm của approach này:
- **Modularity**: Code được tổ chức rõ ràng theo chức năng
- **Reusability**: Có thể tái sử dụng functions trong các project khác
- **Maintainability**: Dễ debug và maintain
- **Collaboration**: Nhiều người có thể làm việc trên các module khác nhau
- **Testing**: Có thể test từng module riêng biệt

### Cách sử dụng:
1. Modify các parameters trong `config.py`
2. Import functions cần thiết vào notebook
3. Chạy từng step hoặc toàn bộ workflow
4. Analyze và visualize kết quả